## Requirements

In [4]:
!python -V

Python 3.6.9


In [ ]:
# Neurokit
!pip install https://github.com/neuropsychology/neurokit/zipball/master

  Using cached https://github.com/neuropsychology/neurokit/zipball/master
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for neurokit2: filename=neurokit2-0.0.42-cp36-none-any.whl size=985198 sha256=3fca82e03fd2828e7f381dab7ee365ffe481a3d548a4e04014b2c69e26df4e42
  Stored in directory: /tmp/pip-ephem-wheel-cache-4tjort4e/wheels/96/b4/4a/5737e93a6740c234539e1fccf52cc0a868c42ae1c667dbefc6
Successfully built neurokit2


In [ ]:
!pip install scipy

In [ ]:
!pip install tsaug

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
based_path = '/content/drive/My Drive/CBS_Nattachai_Thesis Source'


In [ ]:
import pickle
import numpy as np
import random
import pandas as pd

import neurokit2 as nk
import tsaug
from scipy import signal

import matplotlib.pyplot as plt

## Necessary Function

### augmentation

In [ ]:
# peaks_detected
def peaks_detected(ecg, sampling_rate=1000):

  # R peak
  _, rpeaks = nk.ecg_peaks(ecg, sampling_rate=1000)
  # PQST peak
  signals, waves = nk.ecg_delineate(ecg, rpeaks, 
                                    sampling_rate=1000, method='peak')
  # append all peaks
  peaks = {}
  peaks.update(rpeaks)
  peaks.update(waves)
  
  return peaks


# get segment
def get_segments(peaks, p1='ECG_R_Peaks', p2='ECG_T_Peaks'):
  p1_loc = peaks[p1]
  p2_loc = peaks[p2]

  # in case p1_loc[0] > p2_loc[0]
  if p1_loc[0] > p2_loc[0]:
    p1_loc = [int(x) for x in p1_loc[:-1]]
    p2_loc = [int(x) for x in p2_loc[1:]]
  else:
    p1_loc = [int(x) for x in p1_loc]
    p2_loc = [int(x) for x in p2_loc]

  # segment position
  segments = []
  for i,j in list(zip(p1_loc, p2_loc)):
    _seg = list(range(i,j))
    segments.append(_seg)

  return segments



In [ ]:
# augmentation function
# slope
def segment_aug_slope(ecg, segments, frac=0.9):

  new_ecg = ecg.copy()
  # add slope
  for i in range(len(segments)):

    # generate linear slope
    length = int(len(ecg[segments[i]])*frac)
    line = np.linspace(ecg[segments[i]][0]*0.5, 0, num=length)
    
    # add linear to original data
    _aug = new_ecg[segments[i]]
    _aug[_aug.shape[0]-length:] += line


    new_ecg[segments[i]] = _aug
  
  return new_ecg


# flatten
def segment_aug_flatten(ecg, segments):

  new_ecg = ecg.copy()

  # flatten
  for i in range(len(segments)):
    _aug = new_ecg[segments[i]]
    _aug = _aug*0.1

    new_ecg[segments[i]] = _aug

  return new_ecg
  

In [ ]:
# test
#===== setting =====#
duration = 10
sampling_rate = 1000
noise = 0
method = 'ecgsyn' 
heart_rate = np.random.randint(60, 100)

# pattern = pattern
addnoise = 'none'
# n_samples = n_samples

ecg = nk.ecg_simulate(duration=duration, 
                          sampling_rate=sampling_rate, 
                          noise=noise, 
                          heart_rate=heart_rate, 
                          method=method)

# get morphology 
peaks = peaks_detected(ecg, sampling_rate=1000)
# st_segments = get_segments(peaks, p1='ECG_S_Peaks', p2='ECG_T_Peaks')
# tp_segments = get_segments(peaks, p1='ECG_T_Peaks', p2='ECG_P_Peaks') 
# q_peaks = peaks['ECG_Q_Peaks']
# qrs_segments = get_segments(peaks, p1='ECG_Q_Peaks', p2='ECG_S_Peaks')
# rst_segments = get_segments(peaks, p1='ECG_R_Peaks', p2='ECG_T_Peaks')

In [ ]:
np.isnan(peaks['ECG_R_Peaks']).sum()

0

In [ ]:
# ยังมีปัญหา บางครั้ง ecg ที่ simulate มา หา peaks ไม่เจอ >>>> ไว้แก้ให้มัน simulate ใหม่จนกว่าจะเจอ

### generate data

In [ ]:
def generate_ecg_v1(ecg, st_segments, tp_segments, pattern='normal', **kwargs):

  new_ecg = ecg.copy()
  #===== augmentation =====#
  # flatten
  ecg_v1 = segment_aug_flatten(new_ecg, st_segments)
  ecg_v1 = segment_aug_flatten(ecg_v1, tp_segments)

  if pattern == 'normal':
    ecg_v1 = ecg_v1*-1*0.3
    return ecg_v1
  else :
    rst_segments = kwargs.get('rst_segments', None)
    ecg_v1_ste = segment_aug_slope(ecg_v1, segments=rst_segments)
    ecg_v1_ste = ecg_v1_ste*-1*0.3
    return ecg_v1_ste


# ecg_v2 : decrease Q + flip_qrs
def generate_ecg_v2(ecg, q_peaks, qrs_segments, pattern='normal', **kwargs):

  new_ecg = ecg.copy()
  #===== augmentation =====#
  # decrease Q 
  _aug1 = new_ecg[q_peaks]
  _aug1 = _aug1*2
  new_ecg[q_peaks] = _aug1
  # flip_qrs
  for i in range(len(qrs_segments)):
    _aug2 = new_ecg[qrs_segments[i]]
    _aug2 = _aug2*-1

    new_ecg[qrs_segments[i]] = _aug2

  #===== pattern =====#
  if pattern == 'normal':
    ecg_v2 = new_ecg
    return ecg_v2
  else :
    rst_segments = kwargs.get('rst_segments', None)
    ecg_v2 = new_ecg
    ecg_v2_ste = segment_aug_slope(ecg_v2, segments=rst_segments)
    return ecg_v2_ste


# ecg_v3 : decrease Q + flip + scaled
def generate_ecg_v3(ecg, q_peaks, qrs_segments, pattern='normal', **kwargs):

  new_ecg = ecg.copy()
  #===== augmentation =====#
  # decrease Q 
  _aug1 = new_ecg[q_peaks]
  _aug1 = _aug1*2
  new_ecg[q_peaks] = _aug1
  # flip_qrs
  for i in range(len(qrs_segments)):
    _aug2 = new_ecg[qrs_segments[i]]
    _aug2 = _aug2*-1

    new_ecg[qrs_segments[i]] = _aug2
  # scaled
  new_ecg = new_ecg*0.5

  #===== pattern =====#
  if pattern == 'normal':
    ecg_v3 = new_ecg
    return ecg_v3
  else :
    rst_segments = kwargs.get('rst_segments', None)
    ecg_v3 = new_ecg
    ecg_v3_ste = segment_aug_slope(ecg_v3, segments=rst_segments)
    return ecg_v3_ste
  

# ecg_v4 : decrease S + scaled
def generate_ecg_v4(ecg, pattern='normal', **kwargs):

  new_ecg = ecg.copy()
  #===== augmentation =====#
  new_ecg = new_ecg*0.5

  #===== pattern =====#
  if pattern == 'normal':
    ecg_v4 = new_ecg
    return ecg_v4
  else :
    rst_segments = kwargs.get('rst_segments', None)
    ecg_v4 = new_ecg
    ecg_v4_ste = segment_aug_slope(ecg_v4, segments=rst_segments)
    return ecg_v4_ste


### addnoise

In [ ]:
def addnoise_bw(seq, sampling_rate=1000):

  #===== gerate baseline wander noise =====#
  # set parameters (ref. BW Detection Threshold )
  F = random.choice([0.05, 0.1, 0.15, 0.2])   # No. of cycles per second, F = 500 Hz
  T = 10         # Time period, T = 2 ms
  Fs = sampling_rate   # No. of samples per second, Fs = 50 kHz
  Ts = 1./Fs       # Sampling interval, Ts = 20 us
  N = int(T/Ts)     # No. of samples for 2 ms, N = 100
  # generate 
  t = np.linspace(0, T, N)
  noise = np.sin(2*np.pi*F*t)*0.1
  # add noise
  new_seq = seq + noise

  return new_seq


def addnoise_ma(seq):

  # random position
  new_ecg = seq.copy()
  num1 = np.random.randint(int(new_ecg.shape[0]*0.25), int(new_ecg.shape[0]*0.75))
  num2 = np.random.randint(int(new_ecg.shape[0]*0.25), int(new_ecg.shape[0]*0.75))
  _start = min([num1, num2])
  _end = max([num1, num2])

  # generate noise
  noise1 = np.random.randn(new_ecg[:_start].shape[0])*0.02
  #noise2 = np.random.randn(new_ecg[_start:_end].shape[0])*0.02
  noise3 = np.random.randn(new_ecg[_end:].shape[0])*0.02

  # update values
  new_ecg[:_start] += noise1
  #new_ecg[_start:_end] += noise2
  new_ecg[_end:] += noise3

  return new_ecg


def addnoise_gn(seq):

  new_ecg = seq.copy()
  noise = np.random.randn(new_ecg.shape[0])*0.02
  new_ecg += noise

  return new_ecg


## simulate dataset

In [ ]:
# simulate dataset
def generate_ecg_dataset(pattern='normal', addnoise='none', n_samples=10):
  '''
    generate ECG dataset
    pattern : 'normal', 'ste'
    addnoise : 'none', 'bw', 'ma', 'gn'
    n_samples : number of samples
  '''

  #===== setting =====#
  duration = 10
  sampling_rate = 1000
  noise = 0
  method = 'ecgsyn' 
  heart_rate = np.random.randint(60, 100)

  pattern = pattern
  addnoise = 'none'
  n_samples = n_samples

  #===== generate data =====#
  data = []
  for i in range(n_samples):
    ecg = nk.ecg_simulate(duration=duration, 
                          sampling_rate=sampling_rate, 
                          noise=noise, 
                          heart_rate=heart_rate, 
                          method=method)
    # get morphology 
    peaks = peaks_detected(ecg, sampling_rate=1000)
    st_segments = get_segments(peaks, p1='ECG_S_Peaks', p2='ECG_T_Peaks')
    tp_segments = get_segments(peaks, p1='ECG_T_Peaks', p2='ECG_P_Peaks') 
    q_peaks = peaks['ECG_Q_Peaks']
    qrs_segments = get_segments(peaks, p1='ECG_Q_Peaks', p2='ECG_S_Peaks')
    rst_segments = get_segments(peaks, p1='ECG_R_Peaks', p2='ECG_T_Peaks')
    # generate v1-v4
    if pattern == 'normal':
      v1 = generate_ecg_v1(ecg, st_segments, tp_segments, pattern=pattern)
      v2 = generate_ecg_v2(ecg, q_peaks, qrs_segments, pattern=pattern)
      v3 = generate_ecg_v3(ecg, q_peaks, qrs_segments, pattern=pattern)
      v4 = generate_ecg_v4(ecg, pattern=pattern)
    else :
      v1 = generate_ecg_v1(ecg, st_segments, tp_segments, 
                          pattern='ste', rst_segments=rst_segments)
      v2 = generate_ecg_v2(ecg, q_peaks, qrs_segments, 
                          pattern='ste', rst_segments=rst_segments)
      v3 = generate_ecg_v3(ecg, q_peaks, qrs_segments, 
                          pattern='ste', rst_segments=rst_segments)
      v4 = generate_ecg_v4(ecg, pattern='ste', rst_segments=rst_segments)

    smp_data = {'id': i,
                'v1': v1[3001:6001], 
                'v2': v2[3001:6001], 
                'v3': v3[3001:6001], 
                'v4': v4[3001:6001]}

    data.append(smp_data)

  # store data in DataFrame
  df = pd.DataFrame(data)

  # addnoise
  if addnoise == 'none':
    return df

  elif addnoise == 'bw':
    df_bw = df.apply(lambda x: addnoise_bw(x, sampling_rate=1000))
    return df_bw

  elif addnoise == 'ma':
    df_ma = df.apply(lambda x: addnoise_ma(x))
    return df_ma

  elif addnoise == 'gn':
    df_gn = df.apply(lambda x: addnoise_gn(x))
    return df_gn  


In [ ]:
save_path = based_path+'/Deep Learning for TSC/data/'



In [ ]:
ecg = generate_ecg_dataset(pattern='normal', addnoise='none', n_samples=5000)


In [ ]:
ecg.__sizeof__()

2120128

In [ ]:
# save
file = open(save_path+'/ecg5000.pkl', 'wb')
pickle.dump(ecg, file)
file.close()

In [ ]:
# # load
# filehandler = open(save_path+'/ecg5000.pkl', 'rb')
# tmp2 = pickle.load(filehandler)
# filehandler.close()


In [ ]:
ecgste = generate_ecg_dataset(pattern='ste', addnoise='none', n_samples=5000)


In [ ]:
ecgste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   v1      5000 non-null   object
 2   v2      5000 non-null   object
 3   v3      5000 non-null   object
 4   v4      5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [ ]:
# save
file = open(save_path+'/ecgste5000.pkl', 'wb')
pickle.dump(ecgste, file)
file.close()

In [ ]:
ecg_bw = generate_ecg_dataset(pattern='normal', addnoise='bw', n_samples=5000)


In [ ]:
ecg_bw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   v1      5000 non-null   object
 2   v2      5000 non-null   object
 3   v3      5000 non-null   object
 4   v4      5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [ ]:
ecg_bw.loc[0, 'v1'].shape

(3000,)

In [ ]:
# save
file = open(save_path+'/ecg_bw5000.pkl', 'wb')
pickle.dump(ecg_bw, file)
file.close()

In [ ]:
ecgste_bw = generate_ecg_dataset(pattern='ste', addnoise='bw', n_samples=5000)


In [ ]:
ecgste_bw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   v1      5000 non-null   object
 2   v2      5000 non-null   object
 3   v3      5000 non-null   object
 4   v4      5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [ ]:
# save
file = open(save_path+'/ecgste_bw5000.pkl', 'wb')
pickle.dump(ecgste_bw, file)
file.close()


In [ ]:
ecg_ma = generate_ecg_dataset(pattern='normal', addnoise='ma', n_samples=5000)


In [ ]:
ecg_ma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   v1      5000 non-null   object
 2   v2      5000 non-null   object
 3   v3      5000 non-null   object
 4   v4      5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [ ]:
# save
file = open(save_path+'/ecg_ma5000.pkl', 'wb')
pickle.dump(ecg_ma, file)
file.close()


In [ ]:
ecgste_ma = generate_ecg_dataset(pattern='ste', addnoise='ma', n_samples=5000)


In [ ]:
ecgste_ma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   v1      5000 non-null   object
 2   v2      5000 non-null   object
 3   v3      5000 non-null   object
 4   v4      5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [ ]:
ecgste_ma.loc[0, 'v1'].shape

(3000,)

In [ ]:
# save
file = open(save_path+'/ecgste_ma5000.pkl', 'wb')
pickle.dump(ecgste_ma, file)
file.close()


In [ ]:
ecg_gn = generate_ecg_dataset(pattern='normal', addnoise='gn', n_samples=5000)


In [ ]:
ecg_gn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   v1      5000 non-null   object
 2   v2      5000 non-null   object
 3   v3      5000 non-null   object
 4   v4      5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [ ]:
# save
file = open(save_path+'/ecg_gn5000.pkl', 'wb')
pickle.dump(ecg_gn, file)
file.close()


In [ ]:
ecgste_gn = generate_ecg_dataset(pattern='ste', addnoise='gn', n_samples=5000) # ac interference


In [ ]:
ecgste_gn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   v1      5000 non-null   object
 2   v2      5000 non-null   object
 3   v3      5000 non-null   object
 4   v4      5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [ ]:
# save
file = open(save_path+'/ecgste_gn5000.pkl', 'wb')
pickle.dump(ecgste_gn, file)
file.close()


## data preprocessing

In [ ]:
# data preprocessing
def get_input(df):
  '''
    preprocessing data from DataFrame to feed into Neural Network
    the input dimensions should be (m, tx, nx)
    m : number of samples
    tx : number of time points
    nx : number of features
  '''

  data = []
  for i in range(df.shape[0]):
    smp_data = np.vstack(df[['v1', 'v2', 'v3', 'v4']].values[i]).T
    data.append(smp_data)

  data_arr = np.array(data)
  print(data_arr.shape)

  return data_arr


In [ ]:
# load
filehandler = open(save_path+'/ecg_ma5000.pkl', 'rb')
ecg_ma = pickle.load(filehandler)
filehandler.close()


In [ ]:
X = get_input(ecg_ma)

(5000, 3000, 4)


In [ ]:
X[0]

array([[-0.00165872,  0.05529052,  0.02764526,  0.02764526],
       [-0.00176818,  0.05893932,  0.02946966,  0.02946966],
       [-0.00187907,  0.0626355 ,  0.03131775,  0.03131775],
       ...,
       [-0.00265152,  0.08838392,  0.04419196,  0.04419196],
       [-0.00254691,  0.08489701,  0.0424485 ,  0.0424485 ],
       [-0.00244423,  0.08147446,  0.04073723,  0.04073723]])